In [6]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [7]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [8]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

In [9]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [10]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [11]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [12]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links

In [13]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'external company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter/X profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [14]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [15]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 5 relevant links


{'links': [{'type': 'company homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter/X', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [16]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://huggingface.co/join'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'API endpoints', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'Community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'System status', 'url': 'https://status.huggingface.co/'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Discord', 'url': 'https://huggingface.co/join/discord'}]}

In [19]:
## Second step: make the brochure!

# Assemble all the details into another prompt to GPT-5-nano

In [20]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [21]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
moonshotai/Kimi-K2.5
Updated
about 15 hours ago
•
203k
•
1.73k
zai-org/GLM-OCR
Updated
3 days ago
•
96.3k
•
664
Qwen/Qwen3-Coder-Next
Updated
2 days ago
•
18.7k
•
449
stepfun-ai/Step-3.5-Flash
Updated
about 8 hours ago
•
8.69k
•
443
circlestone-labs/Anima
Updated
5 days ago
•
43.1k
•
431
Browse 2M+ models
Spaces
Running
on
Zero
Featured
1.25k
Qwen3-TTS Demo
🎙
1.25k
Transform text into natural-sounding speech with custom voices
Running
449
Demo Playground
⚡
449
Free platform to access multiple AI models
Running
on
Zero
MCP
2.01k
Z Ima

In [22]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

In [23]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [24]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nmoonshotai/Kimi-K2.5\nUpdated\nabout 15 hours ago\n•\n203k\n•\n1.73k\nzai-org/GLM-OCR\nUpdated\n3 days ago\n•\n96.3k\n•\n664\nQwen/Qwen3-Coder-Next\nUpdated\n2 days ago\n•\n18.7k\n•\n449\nstepfun-ai/Step-3.5-Flash\nUpdated\nabout 8 hours ago\n•\n8.69k\n•\n443\ncirclestone-labs/Anima\nUpdated\n5 days ago\n•\n43.1k\n•\n431\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\nFeatured\n1.25k\nQwen3-

In [25]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [26]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the leading AI community and collaboration platform dedicated to building the future of machine learning. It acts as a central hub where machine learning engineers, scientists, and enthusiasts from around the world come together to **create, discover, share, and collaborate** on open-source models, datasets, and AI applications.

Their vision is to empower the next generation of AI creators and users to build an **open and ethical AI future** by providing the tools and community support needed to accelerate innovation.

---

## What Hugging Face Offers

- **Open Collaboration Platform**: Host and work on unlimited public models, datasets, and applications.
- **Massive Repository**: Access over 2 million AI models, 500,000+ datasets, and 1 million+ applications covering modalities like text, images, video, audio, and even 3D.
- **Community-Powered**: A fast-growing vibrant community sharing ready-to-use resources and novel AI innovations.
- **Spaces**: Deploy and showcase custom AI apps such as speech synthesis, image generation, music creation, and more with scalable infrastructure.
- **Accelerated Development**: Use the Hugging Face open-source stack to prototype and build solutions faster than ever.
- **Enterprise Solutions**: Enterprise-grade cloud and compute offerings designed to support teams and organizations wanting robust AI tooling and infrastructure.

---

## Company Culture & Values

Hugging Face embraces a **collaborative, open, and ethical** approach. The company fosters:

- **Community Collaboration**: Encouraging sharing, learning, and co-creation.
- **Openness & Transparency**: Committed to open-source principles and broad accessibility.
- **Innovation & Speed**: Providing tools and infrastructure to rapidly iterate and build AI products.
- **Ethical AI**: Supporting projects and partnerships that promote responsible AI development.

---

## Customers & Users

Hugging Face serves a wide audience including:

- Independent researchers and machine learning engineers.
- Enterprises requiring scalable AI tools and infrastructure.
- Open-source contributors and AI enthusiasts.
- Organizations aiming to integrate cutting-edge AI models in their products.
- Developers building AI-powered applications across industries.

---

## Careers & Opportunities

Join Hugging Face and be part of a dynamic team shaping the future of AI:

- Embedded in a fast-growing startup environment focused on impactful AI technology.
- Work alongside world-class engineers, researchers, and community builders.
- Contribute to open-source projects used by millions globally.
- Competitive roles spanning engineering, research, community engagement, and enterprise solutions.

Interested candidates can explore current openings on their website and become part of this innovative AI journey.

---

## Get Involved

- **Explore AI Apps:** Discover and try thousands of ready-to-use AI applications.
- **Browse Models & Datasets:** Leverage a rich library for your machine learning projects.
- **Share Your Work:** Build your machine learning portfolio and gain visibility.
- **Sign Up:** Join the community for free or opt for enterprise-grade options tailored to your team’s needs.

---

## Contact & Learn More

Visit [huggingface.co](https://huggingface.co) to dive into the world of collaborative machine learning and start building the future with Hugging Face today.

---

**Brand Colors:**  
- Yellow: #FFD21E  
- Orange: #FF9D00  
- Gray: #6B7280  

**Tagline:** *The AI community building the future.*

In [27]:
## Finally - a minor improvement

In [28]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [29]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


# Hugging Face Brochure

---

## About Hugging Face  
Hugging Face is a pioneering AI company and the vibrant community building the future of machine learning. Their platform serves as a dynamic collaboration space where engineers, researchers, and enthusiasts come together to create, discover, share, and experiment with machine learning models, datasets, and applications. Hugging Face empowers the next generation of machine learning practitioners with an open and ethical approach to AI development.

---

## What Hugging Face Offers  

- **Models:** Access to over 2 million open-source machine learning models spanning diverse modalities including text, image, video, audio, and even 3D.  
- **Datasets:** Explore more than 500,000 datasets curated and updated by the community to fuel your ML projects.  
- **Spaces:** Run and share interactive AI applications easily on Hugging Face’s Spaces, including demos for text-to-speech, image generation, music foundation models, and animation tools.  
- **Enterprise Solutions:** Accelerate your team's AI development with enterprise-grade tools, paid compute resources, and advanced platforms designed for scalable innovation.  
- **Open Source Stack:** Built for speed and collaboration, their open-source libraries and tools enable developers to build and deploy models efficiently.

---

## The Community & Culture  
Hugging Face is fundamentally a community-driven platform with a fast-growing global network of machine learning researchers, engineers, and creators. The culture encourages collaboration, openness, and ethical AI development—providing a home where users can build their portfolios, share innovative work, and accelerate machine learning research together. It fosters continuous learning and practical experimentation, ensuring users are empowered to push the boundaries of AI.

---

## Who Uses Hugging Face?  
- **Machine Learning Engineers & Researchers:** For developing, sharing, and discovering cutting-edge models.  
- **Data Scientists & AI Practitioners:** For accessing rich datasets and powerful open-source tools.  
- **Enterprises & Teams:** To leverage scalable compute and enterprise-grade AI infrastructure.  
- **Educators & Students:** Building skills and portfolios in a collaborative environment.  
- **Developers:** Experimenting with AI applications through hosted Spaces and APIs.

---

## Careers at Hugging Face  
Join a mission-driven company dedicated to shaping the future of AI with openness and collaboration. Hugging Face offers opportunities across machine learning engineering, research, product development, community management, and enterprise solutions. Working here means contributing to an open and ethical AI ecosystem and being part of a passionate team that values innovation, diversity, and impact.

---

## Visual Identity  
Hugging Face’s branding features warm, inviting colors such as yellow (#FFD21E) and orange (#FF9D00), complemented by neutral gray (#6B7280). Their logo and design assets are geared towards an accessible, friendly, and community-centric appeal, matching the welcoming nature of the platform.

---

## Get Started with Hugging Face  
- Explore AI models and applications  
- Host and share your own models and datasets  
- Collaborate with a global community of AI creators  
- Access enterprise solutions to scale AI within your organization  
- Build your portfolio and accelerate your machine learning journey  

Visit [huggingface.co](https://huggingface.co) to join the AI community building the future.

---

**Hugging Face — The Home of Machine Learning**  
*Create, discover, and collaborate on ML better.*

In [30]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 15 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is a vibrant and rapidly growing AI community dedicated to building the future of machine learning. It serves as the premier platform where machine learning practitioners, researchers, and enthusiasts come together to collaborate on models, datasets, and applications across diverse modalities such as text, image, video, audio, and even 3D.

The platform empowers users to create, discover, and share innovative machine learning projects, fostering an ecosystem that accelerates ML development and real-world deployment.

---

## What Hugging Face Offers

### Collaboration Platform  
- Host and collaborate on **unlimited public models, datasets, and applications**.  
- Build your professional ML portfolio by sharing your work with a global audience.  
- Access and contribute to over **2 million models**, **500,000+ datasets**, and **1 million+ applications**.

### Open Source Stack  
- Move faster with Hugging Face’s open-source tools and libraries.  
- Benefit from comprehensive documentation and community support to speed up your ML projects.

### Diverse Model and Dataset Ecosystem  
- Explore trending models like Kimi-K2.5, GLM-OCR, and Qwen3-Coder-Next updated regularly by community contributors.  
- Access a vast repository of datasets continuously updated by experts and enthusiasts alike.

### Spaces — AI Applications Hub  
- Discover and launch AI-powered applications hosted on Hugging Face Spaces.  
- Try demo applications for text-to-speech, image generation from text, music generation, animation, and more.

### Enterprise Solutions  
- Tailored paid compute and enterprise-grade platforms support teams and organizations to build AI faster and securely.  
- Scalable infrastructure designed to meet production demands, supporting the transition from research to deployment.

---

## Company Culture

Hugging Face prides itself on being a community-driven organization that values openness, collaboration, and innovation. The company thrives on inclusivity and transparency, encouraging contributions from all levels of expertise to advance machine learning collectively.

They foster a culture where learning and sharing knowledge are core, backed by an active feed of continuous updates, datasets, model improvements, research papers, and articles. Their dedication to open source ensures that the AI community grows stronger by working together.

---

## Who Uses Hugging Face?

- **Researchers and Academics**: Sharing datasets, models, and cutting-edge ideas.  
- **Developers and Data Scientists**: Accessing pre-trained models and building AI applications rapidly.  
- **Enterprises and Teams**: Using enterprise solutions for secure, scalable AI development and deployment.  
- **Artists and Creators**: Leveraging machine learning for creative projects involving text, images, audio, and more.  
- **Machine Learning Enthusiasts**: Engaging with a global community to learn, experiment, and contribute.

---

## Career Opportunities

Hugging Face offers a dynamic workplace where talented individuals can contribute to transformative AI projects that impact millions worldwide. Careers at Hugging Face focus on:

- Engineering advanced ML frameworks and tools.
- Research and development of state-of-the-art AI models.
- Building scalable infrastructure for the AI community.
- Community engagement, documentation, and support roles.

Join Hugging Face to be part of a passionate team shaping the future of AI with openness and collaboration at its core.

---

## Connect with Hugging Face

- **Website:** [https://huggingface.co](https://huggingface.co)  
- **Community Activity:** Stay updated with active contributions and developments through the Hugging Face hub.  

---

*Hugging Face is where the machine learning community builds the future — together.*